Here is just a test notebook.

In [1]:
#import packages 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy import units as u  
import time

In [45]:
#read in the files


I have two files, one from Einstein Observatory and one from ROSAT observatory. They both have detections. Each row in both files are a source detection. I want to see if they could be connected. My first way is to see if they are nearby each other. I want to compare every line in Einstein with every line in ROSAT. I'm trying to do this as fast as possible, or at least a way that isn't taking over 5 days.... these for loops are slow. I tried your np.where suggestion but I still have a for loop and I was wondering if there is a better way to do this.

In [48]:
#Here is one example
#here is all the data. The Einstein file has 46,000 lines and the ROSAT has 112,000 lines
#this takes almost 3 minutes

start = time.time() #start a time to see how long it takes

ROSAT = pd.read_csv("2rxs_lite.txt", delim_whitespace=True)  
Einstein = pd.read_csv("2sig_J2000_theta.txt", delim_whitespace=True)

RA_ROSAT = ROSAT.iloc[:, 2] #right ascention
DEC_ROSAT = ROSAT.iloc[:, 3] #decllination
SNR_ROSAT = ROSAT.iloc[:, 4] #signal to noise ratio

RA_Einstein = Einstein.iloc[:, 0] #right ascention
DEC_Einstein = Einstein.iloc[:, 1] #decllination
SNR_Einstein = Einstein.iloc[:, 2] #signal to noise ratio

count = 0

for i in range(np.size(DEC_Einstein)): # i was wondering, is there a better way to do this instead of a for loop since it is so slow?

    tmp = np.where( ((np.absolute(DEC_Einstein[i] - DEC_ROSAT) < 1)) & ( (np.absolute(RA_Einstein[i]-RA_ROSAT[i]) * np.cos(DEC_ROSAT*(180/np.pi)) )  < 1 ) )  #conditions it need to meet

    if np.size(tmp) ==0: #is there a better way to do this? instead of the if... else statements
            continue

    else: 
        count += 1 #just to see how many matches we get
print(count)


print(f'Numpy: {time.time() - start} seconds')


46185
Numpy: 176.1559317111969 seconds


In [49]:
start = time.time() #start a time to see how long it takes

#Here is the other example with shorter files
#this one only takes 0.7 seconds

ROSAT = pd.read_csv("ROSAT-sorted.txt", delim_whitespace=True)  
Einstein = pd.read_csv("ets_short.txt", delim_whitespace=True)

RA_ROSAT = ROSAT.iloc[:, 2] #right ascention
DEC_ROSAT = ROSAT.iloc[:, 3] #decllination
SNR_ROSAT = ROSAT.iloc[:, 4] #signal to noise ratio

RA_Einstein = Einstein.iloc[:, 0] #right ascention
DEC_Einstein = Einstein.iloc[:, 1] #decllination
SNR_Einstein = Einstein.iloc[:, 2] #signal to noise ratio
count = 0

theta = [] #the angular separation

for i in range(np.size(DEC_Einstein)): # i was wondering, is there a better way to do this instead of a for loop since it is so slow?

    tmp = np.where( ((np.absolute(DEC_Einstein[i] - DEC_ROSAT) < 1)) & ( (np.absolute(RA_Einstein[i]-RA_ROSAT[i]) * np.cos(DEC_ROSAT*(180/np.pi)) )  < 1 ) )  #conditions it need to meet

    if np.size(tmp) ==0: 
            continue
    else: 
        count+= 1
print(count)


print(f'Numpy: {time.time() - start} seconds')


638
Numpy: 0.7407848834991455 seconds


However, I want to do more than just find out how many lines have coordinates that could be near each other so I want to save those coordinates. I took the previous code and am adding to it.

In [52]:
start = time.time() #start a time to see how long it takes

#Here is the other example with shorter files
#this one only takes 0.7 seconds

ROSAT = pd.read_csv("ROSAT-sorted.txt", delim_whitespace=True)  
Einstein = pd.read_csv("ets_short.txt", delim_whitespace=True)

RA_ROSAT = ROSAT.iloc[:, 2] #right ascention
DEC_ROSAT = ROSAT.iloc[:, 3] #decllination
SNR_ROSAT = ROSAT.iloc[:, 4] #signal to noise ratio

RA_Einstein = Einstein.iloc[:, 0] #right ascention
DEC_Einstein = Einstein.iloc[:, 1] #decllination
SNR_Einstein = Einstein.iloc[:, 2] #signal to noise ratio
count = 0

theta = [] #the angular separation

for i in range(np.size(DEC_Einstein)): # i was wondering, is there a better way to do this instead of a for loop since it is so slow?

    tmp = np.where( ((np.absolute(DEC_Einstein[i] - DEC_ROSAT) < 1)) & ( (np.absolute(RA_Einstein[i]-RA_ROSAT[i]) * np.cos(DEC_ROSAT*(180/np.pi)) )  < 1 ) )  #conditions it need to meet

    if np.size(tmp) ==0: 
            continue
    else: 
        for j in range(np.size(tmp)):
                count+= 1
                theta_rad = np.sin(DEC_Einstein[i*180/np.pi])* np.sin(DEC_ROSAT[j*180/np.pi]) + np.cos(DEC_Einstein[i*180/np.pi])*np.cos(DEC_ROSAT[j*180/np.pi])*np.cos(np.absolute(RA_Einstein[i]-RA_ROSAT[i])*180/np.pi) #returns answer in rad
                theta_arcsec = np.arccos(theta_rad) * 206265
                theta.append(theta_arcsec) 
print(count)


print(f'Numpy: {time.time() - start} seconds')


KeyError: 57.29577951308232

In [42]:
L = [i for i in range (1, 10000000) if i%130 == 0]
tmp = [i for i in range (0, np.size(DEC_Einstein)) if np.where( ((np.absolute(tmp_num - DEC_ROSAT) < 1)) & (np.absolute(RA_Einstein-RA_ROSAT) * np.cos(DEC_ROSAT*(180/np.pi))  < 1 ) ) ]
np.size(tmp)

638

In [30]:
print(tmp)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,